In [1]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD 


In [2]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents_banglish.json', 'r', encoding='utf-8').read())

In [3]:
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
print(documents)
    

[(['ওহে'], 'শুভেচ্ছা'), (['কেমন', 'আছো', '?'], 'শুভেচ্ছা'), (['ওখানে', 'কেউ', 'আছো', '?'], 'শুভেচ্ছা'), (['হায়'], 'শুভেচ্ছা'), (['শুভ', 'দিন'], 'শুভেচ্ছা'), (['ভূমি', 'কি', '?'], 'ভূমি'), (['ভূমির', 'সংজ্ঞা'], 'ভূমি'), (['ভূমি', 'কথাটির', 'অর্থ', '?'], 'ভূমি'), (['জমি', 'কি', '?'], 'ভূমি'), (['জমি', 'কথাটির', 'অর্থ', 'কি', '?'], 'ভূমি'), (['জমির', 'সংজ্ঞা', 'কি', '?'], 'ভূমি'), (['vumi', 'ki', '?'], 'ভূমি'), (['jomi', 'ki', '?'], 'ভূমি'), (['vumir', 'songa', 'bolo'], 'ভূমি'), (['ভূমি', 'ধারা'], 'ধারা'), (['ভূমি', 'ধারা', 'কত', 'রকম', '?'], 'ধারা'), (['ভূমি', 'ধারাটি', 'বল'], 'ধারা'), (['vumi', 'dhara', 'ki', '?'], 'ধারা'), (['vumi', 'dhara', 'bolo'], 'ধারা'), (['সম্পদ', 'কি', '?'], 'সম্পদ'), (['সম্পত্তি', 'কি', '?'], 'সম্পদ'), (['সম্পদ', 'বলতে', 'কি', 'বুঝ', '?'], 'সম্পদ'), (['সম্পত্তি', 'বলতে', 'কি', 'বুঝ', '?'], 'সম্পদ'), (['sompod', 'ki', '?'], 'সম্পদ'), (['sompotti', 'ki', '?'], 'সম্পদ'), (['sompod', 'bolte', 'ki', 'bujo', '?'], 'সম্পদ'), (['sompotti', 'bolte', 'ki', 'bujho', '?'],

In [4]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

print(words)

['bolo', 'bolte', 'bujho', 'bujo', 'dhara', 'jomi', 'ki', 'mouja', 'noksha', 'sompod', 'sompotti', 'songa', 'vumi', 'vumir', 'অর্থ', 'আছো', 'ওখানে', 'ওহে', 'কত', 'কথাটির', 'কি', 'কেউ', 'কেমন', 'জমি', 'জমির', 'দিন', 'ধারা', 'ধারাটি', 'নকশা', 'বল', 'বলতে', 'বুঝ', 'ভূমি', 'ভূমির', 'মৌজা', 'রকম', 'শুভ', 'সংজ্ঞা', 'সম্পত্তি', 'সম্পদ', 'হায়']


In [5]:
classes = sorted(set(classes))

In [6]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [7]:
training = []
output_empty = [0]*len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
        
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])
    
    
random.shuffle(training)
training = np.array(training)
    

C:\Users\hp\AppData\Local\Temp\ipykernel_7084\3808730646.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [8]:
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [9]:
model = Sequential()
model.add(Dense(128, input_shape = (len(train_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))

sgd = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])
model.fit(np.array(train_x), np.array(train_y), epochs = 200, batch_size = 5, verbose = 1 )
model.save('Chatbot_model.model')
print('Done')


Epoch 1/200


C:\Users\hp\anaconda3\envs\ILKMS\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


8/8 [==============================] - 0s 2ms/step - loss: 1.8364 - accuracy: 0.1944
Epoch 2/200
8/8 [==============================] - 0s 1ms/step - loss: 1.7167 - accuracy: 0.2222
Epoch 3/200
8/8 [==============================] - 0s 2ms/step - loss: 1.7311 - accuracy: 0.2222
Epoch 4/200
8/8 [==============================] - 0s 2ms/step - loss: 1.6227 - accuracy: 0.3333
Epoch 5/200
8/8 [==============================] - 0s 2ms/step - loss: 1.6185 - accuracy: 0.3333
Epoch 6/200
8/8 [==============================] - 0s 2ms/step - loss: 1.5297 - accuracy: 0.4444
Epoch 7/200
8/8 [==============================] - 0s 2ms/step - loss: 1.4282 - accuracy: 0.4167
Epoch 8/200
8/8 [==============================] - 0s 2ms/step - loss: 1.3750 - accuracy: 0.4444
Epoch 9/200
8/8 [==============================] - 0s 2ms/step - loss: 1.3666 - accuracy: 0.5000
Epoch 10/200
8/8 [==============================] - 0s 2ms/step - loss: 1.3642 - accuracy: 0.4444
Epoch 11/200
8/8 [=======================

8/8 [==============================] - 0s 1ms/step - loss: 0.0250 - accuracy: 1.0000
Epoch 85/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0621 - accuracy: 1.0000
Epoch 86/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0231 - accuracy: 1.0000
Epoch 87/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0620 - accuracy: 1.0000
Epoch 88/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0472 - accuracy: 0.9722
Epoch 89/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0767 - accuracy: 0.9722
Epoch 90/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0527 - accuracy: 1.0000
Epoch 91/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0320 - accuracy: 1.0000
Epoch 92/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0868 - accuracy: 0.9722
Epoch 93/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0094 - accuracy: 1.0000
Epoch 94/200
8/8 [===============

8/8 [==============================] - 0s 2ms/step - loss: 0.0333 - accuracy: 0.9722
Epoch 168/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0320 - accuracy: 1.0000
Epoch 169/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0106 - accuracy: 1.0000
Epoch 170/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0527 - accuracy: 0.9722
Epoch 171/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0247 - accuracy: 1.0000
Epoch 172/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0186 - accuracy: 1.0000
Epoch 173/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0094 - accuracy: 1.0000
Epoch 174/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0062 - accuracy: 1.0000
Epoch 175/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0081 - accuracy: 1.0000
Epoch 176/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0315 - accuracy: 1.0000
Epoch 177/200
8/8 [=====

In [10]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [11]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [12]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    result = [[i, r] for i, r in enumerate(res) if r>ERROR_THRESHOLD]
    result.sort(key=lambda x:x[1], reverse=True)
    
    return_list = []
    
    for r in result:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    
    return return_list

In [13]:
def get_response(intents_list, intents_json):
    if intents_list == []:
        print("Sorry, I didn't get you.")
    else:
        tag = intents_list[0]['intent']
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if i['tag'] == tag:
                result = random.choice(i['response'])
                break
        return result


In [ ]:
print("Go! Bot is running!")

while True:
    message = input("")
    ints = predict_class(message)
    print(ints)
    res = get_response(ints, intents)
    if res == None:
        print("Can you try by something else!!")
    else:
        print(res)

Go! Bot is running!
hi
1/1 [==============================] - 0s 76ms/step
[{'intent': 'শুভেচ্ছা', 'probability': '0.9829181'}]
হায়, স্বাগতম আপনাকে
